# Metric RAG

* median, decile, status (Red / Amber / Green), percentage
* All cost catgeories - cost per pupil
* Utilities and Premises and staff - per m^2

In [1]:
import numpy as np
import pandas as pd
import pickle
import comparator_sets as comparators
import glob 
import os
import time

start_time = time.time()
# Create and clean directory
from pathlib import Path
Path("output/metric-rag").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/metric-rag/*")
for f in files:
    os.remove(f)

## Loading saved comparator sets

In [2]:
with open('output/comparator-sets/schools.pkl', 'rb') as schools_file:
     all_schools = pickle.load(schools_file)

with open('output/comparator-sets/pupil_comparators.pkl', 'rb') as pupil_file:
     pupil_comparators = pickle.load(pupil_file)
     
with open('output/comparator-sets/building_comparators.pkl', 'rb') as building_file:
     building_comparators = pickle.load(building_file)

Declare the RAG mappings

In [3]:
category_settings = {
    'Teaching and Teaching support staff': {
        'type': 'pupil',
        'outstanding_10': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red'],
        'other_10': ['red', 'red', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'other': ['red', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red']
    },
    'Non-educational support staff': {
        'type': 'pupil',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Educational supplies': {
        'type': 'pupil',
        'outstanding_10': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red'],
        'other_10': ['red', 'red', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'other': ['red', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red']
    },
    'IT': {
        'type': 'pupil',
        'outstanding_10': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['amber', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red'],
        'other_10': ['red', 'red', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'red', 'red'],
        'other': ['red', 'amber', 'amber', 'green', 'green', 'green', 'amber', 'amber', 'amber', 'red']
    },
    'Premises': {
        'type': 'area',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Utilities': {
        'type': 'area',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Administrative supplies': {
        'type': 'pupil',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Catering staff and supplies': {
        'type': 'pupil',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    },
    'Other costs': {
        'type': 'pupil',
        'outstanding_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'outstanding': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other_10': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red'],
        'other': ['green', 'green', 'green', 'amber', 'amber', 'amber', 'amber', 'amber', 'red', 'red']
    }
}


def is_area_close_comparator(y, x):
    return (((x['Total Internal Floor Area'] * 0.9) >= y['Total Internal Floor Area']) | (y['Total Internal Floor Area'] <= (x['Total Internal Floor Area'] * 1.1)) 
             & ((x['Age Average Score'] * 0.75) >= y['Age Average Score']) | (y['Age Average Score'] <= (x['Age Average Score'] * 1.25)))

def is_pupil_close_comparator(y, x):
    return (((x['Number of pupils'] * 0.75) >= y['Number of pupils']) | (y['Number of pupils'] <= (x['Number of pupils'] * 1.25)) 
             & ((x['Percentage Free school meals'] * 0.95) >= y['Percentage Free school meals']) | (y['Percentage Free school meals'] <= (x['Percentage Free school meals'] * 1.05))
             & ((x['Percentage SEN'] * 0.90) >= y['Percentage SEN']) | (y['Percentage SEN'] <= (x['Percentage SEN'] * 1.10))
            )

def is_close_comparator(t, y, x):
    if t == 'area': 
        return is_area_close_comparator(y, x)
    else: 
        return is_pupil_close_comparator(y, x)

def map_rag(d, ofstead, rag_mapping):
    close_count = d['is_close'][d['is_close']].count()
    key = 'outstanding' if ofstead.lower() == 'outstanding' else 'other'
    key += '_10' if close_count > 10 else ''
    d['rag'] = d['decile'].fillna(0).map(lambda x: rag_mapping[key][int(x)])
    return d    


Select the school for the RAG calculation

In [4]:
target_school = 103341
comparator_set = comparators.get_comparator_set_by(lambda s: s.index == target_school, all_schools, pupil_comparators).set_index('URN')

Compute the rag calculation for the cost categories given in the [Benchmarking cost categories doc](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/DfEFinancialBenchmarking/_layouts/15/Doc.aspx?sourcedoc=%7B622FB0F9-7CB1-445A-8FFA-664F8857F036%7D&file=Benchmarking%20cost%20categories%20and%20sub-categories.docx&action=default&mobileredirect=true)

In [5]:
base_cols = ['URN', 'Total Internal Floor Area','Age Average Score', 'OfstedRating (name)','Percentage SEN', 'Percentage Free school meals', 'Number of pupils']

result = {}
for category in category_settings:
    deciles = category_settings[category]
    cols = (comparator_set.columns.isin(base_cols) | comparator_set.columns.str.startswith(category))
    
    df = comparator_set[comparator_set.columns[cols]].copy()
    target = comparator_set[comparator_set.index == target_school][comparator_set.columns[cols]].copy()
    
    if deciles['type'] == 'area':
        df['Total'] = df[df.columns[6:df.shape[1]]].sum(axis=1) / (df['Total Internal Floor Area'])
    else:
        df['Total'] = df[df.columns[6:df.shape[1]]].sum(axis=1) / (df['Number of pupils'])
        
    df['is_close'] = df.apply(lambda x: is_close_comparator(deciles['type'], target, x), axis=1)
    df['decile'] = pd.qcut(df['Total'], 10, labels=False, duplicates='drop')
    
    result[category] = map_rag(df, target['OfstedRating (name)'].values[0], deciles).sort_values(by='decile', ascending=True)
    

## Processing Time

In [6]:
print(f'Processing Time: {time.time() - start_time} seconds')

Processing Time: 2.3098113536834717 seconds
